# <center>Recupération des données </center>
# <center>et création d'un dataset </center>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
import re
import time
import sys
import lxml.html
from datetime import datetime
NB_Pages=2#2032
liensBD=[]

In [2]:
def heure():
    H=datetime.now().time().hour
    M=datetime.now().time().minute
    return H,M

In [3]:
# Critiques de BD:
for idx in range (1, NB_Pages):
    if (idx%800!=0 or idx==1341):
        dom = lxml.html.fromstring(requests.get('https://www.babelio.com/livres-/bande-dessinee/18?page='+str(idx)).content)
        liensBD+=[x for x in dom.xpath('//a/@href') if '/livres/' in x and ('#c') not in x]
        Heure=heure()
        print(Heure[0],":",Heure[1],"Pages ",str(idx)," sur ",str(NB_Pages))
        time.sleep(9)
    else :
        time.sleep(600)

22 : 7 Pages  1  sur  2


In [4]:
dfl = pd.DataFrame(liensBD)
dfl=dfl.rename(columns = {0:'liens'})
# On garde une copie des liens récupérer sur le site babelio.com
# On ne sauvegrde que la partie
dfl.to_csv('CSV_tmp/list.csv')

In [5]:
dfl=pd.read_csv('CSV_tmp/list.csv')
dfl

,Unnamed: 0,liens
0,0,/livres/Herge-Les-Aventures-de-Tintin-tome-03-...
1,1,/livres/Herge-Les-Aventures-de-Tintin-tome-11-...
2,2,/livres/Herge-Les-Aventures-de-Tintin-tome-17-...
3,3,/livres/Herge-Les-aventures-de-Tintin-Tome-6--...
4,4,/livres/Herge-Les-aventures-de-Tintin-tome-16-...
...,...,...
95,95,/livres/Mermoux-Linvitation/892315
96,96,/livres/Franquin-Spirou-et-Fantasio-tome-2--Il...
97,97,/livres/Fabcaro-Zai-zai-zai-zai/719553
98,98,/livres/Davodeau-Les-ignorants/295525


In [6]:
dfl=dfl.drop('Unnamed: 0',axis=1)

In [7]:
dfl

,liens
0,/livres/Herge-Les-Aventures-de-Tintin-tome-03-...
1,/livres/Herge-Les-Aventures-de-Tintin-tome-11-...
2,/livres/Herge-Les-Aventures-de-Tintin-tome-17-...
3,/livres/Herge-Les-aventures-de-Tintin-Tome-6--...
4,/livres/Herge-Les-aventures-de-Tintin-tome-16-...
...,...
95,/livres/Mermoux-Linvitation/892315
96,/livres/Franquin-Spirou-et-Fantasio-tome-2--Il...
97,/livres/Fabcaro-Zai-zai-zai-zai/719553
98,/livres/Davodeau-Les-ignorants/295525


In [8]:
liensBD=dfl['liens']

In [9]:
def Changer_type(l, dtype=str):
    return list(map(dtype, l))


def remove_tags(text) :
    TAG_RE=re.sub('\n |\t|\n', '', str(text))
    return TAG_RE

In [10]:
l = [[0 for i in range(27)] for i in range(len(liensBD))]
idx=0
df=pd.DataFrame(l,columns={'Liens':"",'EAN':"",'Titre':"",'Genre(s)':"",'Resume':"",'Image':"","Rating" : "",'Auteur_1':"",'Dessinateur_1':"",
                           "L1":"","N1":"","Critique_1":"",
                           "L2":"","N2":"","Critique_2":"",
                           "L3":"","N3":"","Critique_3":"",
                           "L4":"","N4":"","Critique_4":"",
                           "L5":"","N5":"","Critique_5":"",
                           "L6":"","N6":"","Critique_6":""},dtype='object')


In [15]:
# POUR LE POC ON SE CONCENTRE SUR L'AUTEUR PRINCIPAL
for url_BD in liensBD[14:16]:  
        idx+=1
        LinksList=[]
        Heure=heure()
        print(Heure[0],":",Heure[1],"|| album numéro ",str(idx) , ": ",str(url_BD))
        if idx%150 or idx==0:
                try:
                    response_BD=requests.post("https://www.babelio.com/"+url_BD)
                    soup = BeautifulSoup(response_BD.text)
                    EAN=soup.find('div',class_="livre_refs grey_light").text
                    EAN=EAN[9:23]
                    #print(EAN)
                    Title=soup.find("h1", itemprop="name").text
                    Title=remove_tags(Title)
                    #print(Title)
                    Image=soup.find("div",class_="col col-4")
                    img_src = Image.find('img').attrs['src']
                    Auteur1=soup.find("span", itemprop="name").text
                    #print(Auteur1)
                    Dessinateur=""
                    Description=[des for des in (soup.find("div", itemprop="description",class_="livre_resume")) if des!= '[]']
                    Description=remove_tags(Description)
                    #print(Description)
                    Rating=soup.find("span", itemprop="ratingValue").text
                    #print(Rating)
                    Critiques=[critique.text for critique in  soup.find_all("div", id=re.compile('^cri_'))]
                    Lst_Lecteurs=[lecteur.text for lecteur in soup.find_all("span", itemprop="name")] 
                    #print(Lst_Lecteurs)
                    print(Notes)
                    p=re.compile('^Value')
                    LstNote=[p for note in (p)]
                    print(LstNote)
                    Genres=[Gen.text for Gen in (soup.find_all("a",class_="tag_t14"))]
                    #print(Genres)
                    #tmp=[[0 for i in range(1-len(Critiques))] for i in range(len(liensBD))]
                    df.loc[idx-1]=[idx,EAN,Title,Genres,Description,img_src,Rating,Auteur1,Dessinateur,"","","","","","","","","","","","","","","","","",""]
                    jdx=11
                    i=0
                    lcr=len(Critiques)
                    llec=len(Lst_Lecteurs)
                    for critique in Critiques:
                        df.loc[idx-1][jdx]=remove_tags(critique)
                        df.loc[idx-1][jdx-2]=Lst_Lecteurs[i+(llec-lcr)]
                        jdx+=3
                        i+=1
                        
                    time.sleep(9)
                except:
                    print("Erreur sur le livre ",str(url_BD))
        else:
            df.to_csv('CSV_tmp/BBD.csv')
            print("Sauvegarde tmp...")
            time.sleep(600)
        
        

22 : 11 || album numéro  3 :  /livres/Herge-Les-aventures-de-Tintin-tome-20--Tintin-au-Tibet/3941
[]
Erreur sur le livre  /livres/Herge-Les-aventures-de-Tintin-tome-20--Tintin-au-Tibet/3941
22 : 11 || album numéro  4 :  /livres/Herge-Les-Aventures-de-Tintin-tome-02--Tintin-au-Congo/401916
[]
Erreur sur le livre  /livres/Herge-Les-Aventures-de-Tintin-tome-02--Tintin-au-Congo/401916


In [31]:
pd.set_option('display.max_columns', None)
df.head()

,Liens,EAN,Titre,Genre(s),Resume,Image,Rating,Auteur_1,Dessinateur_1,L1,N1,Critique_1,L2,N2,Critique_2,L3,N3,Critique_3,L4,N4,Critique_4,L5,N5,Critique_5,L6,N6,Critique_6
0,1,9782014001259,"Astérix, tome 6 : Astérix et Cléopâtre",[ historique ],"[""\n\t\t\t\t\t\t\t\t\t\tNous sommes en 50 avan...",https://m.media-amazon.com/images/I/513iIeNEQx...,4.2,René Goscinny,,Nastasia-B,,"QUI PEUT REMPLACER GOSCINNY ?J'en vois, par le...",FrancoMickey,,"Non, je ne me suis pas planté dans l'ordre de ...",Gwen21,,Sans aucun doute l'un de mes plus beaux souven...,XanderOne,,S'il existe un album culte dans la série d'Ast...,domisylzen,,Pauvre Numérobis ! le voilà convoqué par la re...,,,
1,2,9782203001015,"Les Aventures de Tintin, tome 02 : Tintin au C...","[ milou , ligne claire ]","['\n\t\t\t\t\t\t\t\t\t\tLe 10 janvier 1929, un...",/couv/bm_5953_766334.jpg,3.32,Hergé,,bibiouest,,Sur Tintin énormément de choses ont été dites ...,GeorgesSmiley,,"Sans doute ma première vraie lecture, quelque ...",ancoline,,Je voulais le lire pour comprendre ces discuti...,ChatDuCheshire,,Dans l'absolu j'aurais plutôt mis trois étoile...,fabienne1809,,"C'est une de mes premières lectures, si pas la...",,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv('CSV/BabelioLstBD.csv', sep='\t')